In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
from __future__ import unicode_literals

import os
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Introduction

On fait des plots sur les runs

In [ ]:
from problem.workflow import print

# Data

In [ ]:
from problem.mnist import load_data

In [ ]:
X, y = load_data()

# Load models

In [ ]:
from problem.mnist import get_save_directory
from problem.mnist import preprocessing
from problem.mnist import skew
from problem.mnist import tangent
from main import get_data_shape

In [ ]:
def load_models(get_model, *args, **kwargs):
    models = []
    n_features, n_classes = get_data_shape('mnist')
    for i in range(12):
        model = get_model(*args, n_features=n_features, n_classes=n_classes, **kwargs)
        save_directory = get_save_directory()
        model_name = '{}-{}'.format(model.get_name(), i)
        path = os.path.join(save_directory, model_name)
        model.load(path)
        models.append(model)
    return models

In [ ]:
from myNN import get_model as get_model_NN
from myNNDA import get_model as get_model_NNDA
from myTP import get_model as get_model_TP
from myPAN import get_model as get_model_PAN


# Test submission

In [ ]:
from problem.mnist import test_submission


In [ ]:
def get_mean_std(df_list):
    z = df_list[0]['z']
    acc = np.concatenate([df['accuracy'].values.reshape(-1, 1) for df in df_list], axis=1)
    mean_acc = np.mean(acc, axis=1)
    std_acc = np.std(acc, axis=1)
    return mean_acc, std_acc

In [ ]:
# z_list = list(range(-50, 51, 10)) + list(range(-5, 6, 1))
# z_list = sorted(z_list)
z_list = np.linspace(-10, 10, num=21)


In [ ]:
print(z_list)

In [ ]:
PAN_L0_models = load_models(get_model_PAN, preprocessing=preprocessing, skew=skew, trade_off=0., width=5.)
df_list = test_submission(PAN_L0_models, X, y, z_list=z_list)
PAN_L0_mean_acc, PAN_L0_std_acc = get_mean_std(df_list)


In [ ]:
PAN_L01_models = load_models(get_model_PAN, preprocessing=preprocessing, skew=skew, trade_off=0.1, width=5.)
df_list = test_submission(PAN_L01_models, X, y, z_list=z_list)
PAN_L01_mean_acc, PAN_L01_std_acc = get_mean_std(df_list)


In [ ]:
PAN_L1_models = load_models(get_model_PAN, preprocessing=preprocessing, skew=skew, trade_off=1.0, width=5.)
df_list = test_submission(PAN_L1_models, X, y, z_list=z_list)
PAN_L1_mean_acc, PAN_L1_std_acc = get_mean_std(df_list)


In [ ]:
PAN_L10_models = load_models(get_model_PAN, preprocessing=preprocessing, skew=skew, trade_off=10.0, width=5.)
df_list = test_submission(PAN_L10_models, X, y, z_list=z_list)
PAN_L10_mean_acc, PAN_L10_std_acc = get_mean_std(df_list)


In [ ]:
PAN_L100_models = load_models(get_model_PAN, preprocessing=preprocessing, skew=skew, trade_off=100.0, width=5.)
df_list = test_submission(PAN_L100_models, X, y, z_list=z_list)
PAN_L100_mean_acc, PAN_L100_std_acc = get_mean_std(df_list)


# Plot


- What about confusion matrices ???


In [ ]:
sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")

In [ ]:
z = z_list

plt.figure(figsize=(8*2,6*2))

plt.plot(z, PAN_L0_mean_acc, 'o-', label='0.1')
# plt.fill_between(z, PAN_L0_mean_acc+PAN_L0_std_acc, PAN_L0_mean_acc-PAN_L0_std_acc, alpha=0.5)

plt.plot(z, PAN_L01_mean_acc, 'o-', label='0.1')
# plt.fill_between(z, PAN_L01_mean_acc+PAN_L01_std_acc, PAN_L01_mean_acc-PAN_L01_std_acc, alpha=0.5)

plt.plot(z, PAN_L1_mean_acc, 'o-', label='1')
# plt.fill_between(z, PAN_L1_mean_acc+PAN_L1_std_acc, PAN_L1_mean_acc-PAN_L1_std_acc, alpha=0.5)

plt.plot(z, PAN_L10_mean_acc, 'o-', label='10')
# plt.fill_between(z, PAN_L10_mean_acc+PAN_L10_std_acc, PAN_L10_mean_acc-PAN_L10_std_acc, alpha=0.5)

plt.plot(z, PAN_L100_mean_acc, 'o-', label='100')
# plt.fill_between(z, PAN_L100_mean_acc+PAN_L100_std_acc, PAN_L100_mean_acc-PAN_L100_std_acc, alpha=0.5)


plt.xlabel('angle')
plt.ylabel('accuracy')
plt.legend()
plt.show()


In [ ]:
# plt.figure(figsize=(15,8))

zz = np.random.normal( loc=0, scale=5, size=(X.shape[0]) )
sns.distplot(zz, label='5')

zz = np.random.normal( loc=0, scale=10, size=(X.shape[0]) )
sns.distplot(zz, label='10')

zz = np.random.normal( loc=0, scale=15, size=(X.shape[0]) )
sns.distplot(zz, label='15')

plt.xlabel('angle')
plt.legend()
plt.show()

# Playground

In [ ]:
# i = 0
# m = models[i]
# cv_iter = get_cv_iter(X, y)
# idx_dev, idx_valid = cv_iter[i]
# X_test = X[idx_valid]
# y_test = y[idx_valid]
# acc = accuracy_score(y_test, m.predict(X_test))

# print(acc)


In [ ]:
# XX = skew(X_test, z=80)
# y_pred = m.predict(XX)
# acc = accuracy_score(y_test, m.predict(XX))
# print(acc)

# i = np.random.randint(0, XX.shape[0])
# plt.imshow(XX[i].reshape(28,28))

# plt.title('label = {}<->truth = {}'.format(y_pred[i], y_test[i]))
# plt.show()

# plt.imshow(X_test[i].reshape(28,28))
# plt.show()


In [ ]:
# X_test.shape

In [ ]:
# from myNNDA import data_augment

In [ ]:
# XX = X[:200]
# yy = y[:200]
# XX2, yy, _ = data_augment(XX, yy, None)

In [ ]:
# i = 67
# x = XX2[i].reshape(28, 28)
# plt.imshow(x, cmap='Greys')
# plt.title('skewed')
# plt.show()
# x = XX[i].reshape(28, 28)
# plt.imshow(x, cmap='Greys')
# plt.title('original')
# plt.show()